# TITLE

## Introduction

* Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
* Clearly state the question you will try to answer with your project
* Identify and describe the dataset that will be used to answer the question

## Preliminary exploratory data analysis

* Demonstrate that the dataset can be read from the web into R 
* Clean and wrangle your data into a tidy format
* Using only **training data**, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
* Using only **training data**, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

## Methods

* Explain how you will conduct either your data analysis and which variables/columns you will use. *Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?*
* Describe at least one way that you will visualize the results

## Expected outcomes and significance

* What do you expect to find?
* What impact could such findings have?
* What future questions could this lead to?

In [33]:
library(tidyverse)
library(tidymodels)
library(repr)

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3      ✔ yardstick 0.0.7 

Warning message:
“package ‘broom’ was built under R version 4.0.2”
Warning message:
“package ‘dials’ was built under R version 4.0.2”
Warning message:
“package ‘infer’ was built under R version 4.0.3”
Warning message:
“package ‘modeldata’ was built under R version 4.0.1”
Warning message:
“package ‘parsnip’ was built under R version 4.0.2”
Warning message:
“package ‘recipes’ was built under R version 4.0.1”
Warning message:
“package ‘tune’ was built under R version 4.0.2”
Warning message:
“package ‘workflows’ was built under R version 4.0.2”
Warning message:
“package ‘yardstick’ was built under R version 4.0.2”
── Conflicts ────

**Step 1: tidying the data**

In [41]:
player_stats <- read_csv("player_stats.csv")

names(player_stats) <- gsub(" ", "_", names(player_stats)) %>%
                        tolower()

tidy_player_stats <- player_stats %>%
                        separate(age, sep = " ", into = c("age", "birthdate"), convert = TRUE) %>%
                        separate(current_rank, sep = " ", into = c("current_rank", "current_rank_2"), convert = TRUE) %>%
                        separate(best_rank, sep = " ", into = c("best_rank", "best_rank_date"), convert = TRUE) %>%
                        separate(best_rank_date, sep = "-", into = c("best_rank_day", "best_rank_month", "best_rank_year")) %>%
                        mutate(current_rank_2 = parse_number(current_rank_2), best_rank_year = parse_number(best_rank_year)) %>%
                        select(-x1, -wikipedia, -name, -facebook, -twitter, -nicknames, -birthdate, -coach, -web_site, -best_rank_day, -best_rank_month) %>%
                        arrange(current_rank) %>%
                        filter(!is.na(current_rank))

head(tidy_player_stats)



Warning message:
“Missing column names filled in: 'X1' [1]”
Parsed with column specification:
cols(
  .default = col_character(),
  X1 = col_double(),
  `Turned Pro` = col_double(),
  Seasons = col_double(),
  Titles = col_double(),
  `Best Season` = col_double(),
  Retired = col_double(),
  Masters = col_double(),
  `Grand Slams` = col_double(),
  `Davis Cups` = col_double(),
  `Team Cups` = col_double(),
  Olympics = col_double(),
  `Weeks at No. 1` = col_double(),
  `Tour Finals` = col_double()
)

See spec(...) for full column specifications.



age,country,plays,current_rank,current_rank_2,best_rank,best_rank_year,backhand,prize_money,height,⋯,masters,birthplace,residence,weight,grand_slams,davis_cups,team_cups,olympics,weeks_at_no._1,tour_finals
<int>,<chr>,<chr>,<int>,<dbl>,<int>,<dbl>,<chr>,<chr>,<chr>,⋯,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
33,Spain,Left-handed,1,10235,1,2008,Two-handed,"US$119,601,561",185 cm,⋯,35,"Manacor, Mallorca, Spain","Manacor, Mallorca, Spain",85 kg,19,5,NA,1,208,NA
32,Serbia,Right-handed,2,9720,1,2011,Two-handed,"US$ 139,144,944 All-time leader in earnings",188 cm,⋯,34,"Belgrade, Serbia","Monte Carlo, Monaco",77 kg,16,1,1,NA,275,5
38,Switzerland,Right-handed,3,6590,1,2004,One-handed,"$ 129,231,891 2nd all-time leader in earnings",185 cm,⋯,28,"Basel, Switzerland","Bottmingen, Switzerland",85 kg,20,1,NA,NA,310,6
23,Russian Federation,Right-handed,4,5960,4,2019,Two-handed,"US$ 10,507,693",NA,⋯,2,NA,NA,NA,NA,NA,NA,NA,NA,NA
26,Austria,Right-handed,5,5890,4,2017,One-handed,"$22,132,368 15th all-time leader in earnings",185 cm,⋯,1,"Wiener Neustadt, Austria","Lichtenworth, Austria",82 kg,NA,NA,NA,NA,NA,NA
21,Greece,Right-handed,6,5375,5,2019,One-handed,"$10,425,605",NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,1


**Step 2: Splitting into training and test data**

In [42]:
set.seed(1)

stats_split <- initial_split(tidy_player_stats, prop = 0.75, strata = current_rank)
stats_training <- training(stats_split)
stats_testing <- testing(stats_split)